In [37]:
import tensorflow as tf
import numpy as np
%config Completer.use_jedi = False
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
from sklearn.model_selection import train_test_split
# %config IPCompleter.greedy=True

In [13]:
tok=tf.keras.preprocessing.text.Tokenizer(char_level=True)

In [14]:
tok.fit_on_texts(texts)

In [15]:
texts=[
    "where mlflow_bucket is a S3 bucket that have been priorly created. ",
   " Here you would ask “how the hell does MLflow access to my S3 bucket ?",
   " Well, simply read the documention hehe",
    
    
]

In [ ]:
!pip3 install jupyter-tabnine
!jupyter nbextension install --py jupyter_tabnine
!jupyter nbextension enable --py jupyter_tabnine
!jupyter serverextension enable --py jupyter_tabnine

In [16]:
X=tok.texts_to_sequences(texts)

In [17]:
X_padded=tf.keras.preprocessing.sequence.pad_sequences(X,maxlen=100)

In [18]:
max(tok.word_index.values())+1

29

In [19]:
tok.word_counts

OrderedDict([('w', 6),
             ('h', 10),
             ('e', 22),
             ('r', 6),
             (' ', 32),
             ('m', 5),
             ('l', 11),
             ('f', 2),
             ('o', 10),
             ('_', 1),
             ('b', 4),
             ('u', 6),
             ('c', 7),
             ('k', 4),
             ('t', 10),
             ('i', 4),
             ('s', 8),
             ('a', 7),
             ('3', 2),
             ('v', 1),
             ('n', 3),
             ('p', 2),
             ('y', 4),
             ('d', 5),
             ('.', 1),
             ('“', 1),
             ('?', 1),
             (',', 1)])

In [38]:
df=pd.read_csv('spam.csv',error_bad_lines=False,encoding= 'Windows-1252')
X=df['v2']
y=df['v1'].apply(lambda x:0 if x=='spam' else 1)
X_train,X_test,y_train,y_test =train_test_split(X,y,test_size=0.25)

In [39]:
num_words=None
embedding_length=20
tok=Tokenizer(num_words=num_words)
tok.fit_on_texts(X_train)
X_train=tok.texts_to_sequences(X_train)
X_test=tok.texts_to_sequences(X_test)
X_train=pad_sequences(X_train)
X_test=pad_sequences(X_test,maxlen=X_train.shape[1])
y_train=y_train.to_numpy()
y_test=y_test.to_numpy()

In [40]:
maxsequencelength=X_train.shape[1]
maxsequencelength

189

In [41]:
input_dim=num_words if num_words else (max(tok.word_index.values())+1)
input_dim

7753

In [42]:
inputlayer=tf.keras.layers.Input(shape=[maxsequencelength,])

In [43]:
x=tf.keras.layers.Embedding(input_dim=input_dim,output_dim=embedding_length,input_length=maxsequencelength)(inputlayer)
x=tf.keras.layers.LSTM(15, return_sequences=True)(x)
x=tf.keras.layers.GlobalMaxPooling1D()(x)
x=tf.keras.layers.Dense(1,activation='sigmoid')(x)

In [44]:
mymodel=tf.keras.Model(inputs=inputlayer,outputs=x)

In [45]:
mymodel.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 189)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 189, 20)           155060    
_________________________________________________________________
lstm_1 (LSTM)                (None, 189, 15)           2160      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 15)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 16        
Total params: 157,236
Trainable params: 157,236
Non-trainable params: 0
_________________________________________________________________


In [47]:
mymodel.compile(loss='binary_crossentropy',metrics=['accuracy'])

In [48]:
r = mymodel.fit(x=X_train, y=y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
131/131 [==============================] - 10s 62ms/step - loss: 0.4473 - accuracy: 0.8562 - val_loss: 0.3416 - val_accuracy: 0.8952
Epoch 2/10
131/131 [==============================] - 8s 63ms/step - loss: 0.4036 - accuracy: 0.8562 - val_loss: 0.3232 - val_accuracy: 0.8952
Epoch 3/10
131/131 [==============================] - 9s 69ms/step - loss: 0.3292 - accuracy: 0.8562 - val_loss: 0.2311 - val_accuracy: 0.8952
Epoch 4/10
131/131 [==============================] - 8s 62ms/step - loss: 0.2266 - accuracy: 0.8772 - val_loss: 0.1716 - val_accuracy: 0.9526
Epoch 5/10
131/131 [==============================] - 8s 64ms/step - loss: 0.1612 - accuracy: 0.9653 - val_loss: 0.1267 - val_accuracy: 0.9698
Epoch 6/10
131/131 [==============================] - 8s 63ms/step - loss: 0.1091 - accuracy: 0.9794 - val_loss: 0.0950 - val_accuracy: 0.9734
Epoch 7/10
131/131 [==============================] - 8s 61ms/step - loss: 0.0822 - accuracy: 0.9809 - val_loss: 0.0800 - val_accuracy: 0.974

In [27]:
mymodel.compile('rmsprop', 'mse')
mymodel.predict(X_padded)

array([[[ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        [ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        [ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        ...,
        [-0.03317225,  0.02501747, -0.01902874, ..., -0.03911977,
          0.03258922,  0.01445447],
        [ 0.04698439, -0.03220592, -0.02418882, ...,  0.02985627,
          0.0397631 ,  0.02810157],
        [-0.0069503 ,  0.01414237,  0.0128042 , ..., -0.04951246,
          0.02996159,  0.0051683 ]],

       [[ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        [ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        [ 0.01894981,  0.01734537, -0.00833262, ...,  0.0161388 ,
          0.03657594, -0.00620872],
        ...,
        [-0.00737869,  0.04723985, -0.03126812, ..., -

In [30]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(max(tok.word_index.values())+1, 64, input_length=100))

input_array = np.random.randint(1000, size=(32, 10))
model.compile('rmsprop', 'mse')
model.fit(X_padded)
# print(output_array.shape)

ValueError: in user code:

    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.6/dist-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.6/dist-packages/keras/engine/training.py:835 run_step  **
        outputs = model.train_step(data)
    /usr/local/lib/python3.6/dist-pack

In [27]:
pretrained_weights=model.layers[0].get_weights()[0]

IndexError: list index out of range